### IMPORTS

In [212]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from collections import Counter
import pprint
import sys
import spotipy.util as util
from random import shuffle

### GLOBAL VARIABLES

In [214]:
client_ID = "bef34a7d63ba4ace8e7dc8c05de24591"

with open("secret.txt") as file:
    client_secret = file.read()[:-1]
    
sp = spotipy.Spotify(
    client_credentials_manager = SpotifyClientCredentials(
        "{}".format(client_ID), "{}".format(client_secret)))

host = "m11oiwplfr6aqk5svov48b40w"

In [215]:
# sedepa//zltm3cqxt0u0s9k4e8oy564dk

genres = []

users = input("Enter the members' Spotify usernames, separated by '//' ").split("//")
print(users)

Enter the members' Spotify usernames, separated by '//' sedepa//zltm3cqxt0u0s9k4e8oy564dk
['sedepa', 'zltm3cqxt0u0s9k4e8oy564dk']


### OBTAIN USERS' PLAYLISTS

In [216]:
def get_user_playlists():
    
    users_playlists_names = {}
    users_playlists_names_IDs = {}
    
    for user in users:
        playlists = sp.user_playlists(user)
        playlists_names = []
        users_playlists_names_IDs[user] = {}
        
        for i, playlist in enumerate(playlists["items"]):
            playlists_names.append(playlist["name"])
            users_playlists_names_IDs[user][playlist["name"]] = playlist['uri'][17:]
            
        users_playlists_names[user] = playlists_names
        
    
    mygroup = {"users": {}}
    for user in users:
        mygroup["users"][user] = {} 
        
        
        
    for user in users:
        mygroup["users"][user]["playlists"] = input(
            """
            ·················································
            
            {} , enter the Spotify playlists you would like to upload, separated by '//'. 
                        
            These are all your playlists: 
    
            {} 
            
            
            If you want to submit all, just write "all"
                        
            """
            .format(user, users_playlists_names[user])).split("//")
    
        mygroup["users"][user]["genres"] = input(
            """
            ·················································
            
            {} , now select the genres of the songs for your new playlist.
                        
            You can select out of the following:
    
            {} 
            
            
            If you don't want to filter by genre, just write "any"
            
            Again, separate by '//'
                        
            """
            .format(user, genres)).split("//")


    mygroup["new_playlist_name"] = input(
            """
            ·················································
            
            And finally, write the name for your new playlist :D            
            
            """
            )
    
        
    return users_playlists_names_IDs, mygroup

### OBTAIN TRACKS INFO

In [217]:
def get_tracks_info(users_playlists_names_IDs, mygroup):
    users_playlists_tracks = {}
    for user in mygroup["users"].keys():
        tracks_info = []
        
        if mygroup["users"][user]["playlists"] == ["all"]:
            for playlist_ID in users_playlists_names_IDs[user].values():
                
                tracks_info.append(sp.user_playlist_tracks(user, playlist_id = playlist_ID))
        
        else:
            for playlist in mygroup["users"][user]["playlists"]:
                tracks_info.append(sp.user_playlist_tracks(user, 
                                                           playlist_id = users_playlists_names_IDs[user][playlist]))
    
        users_playlists_tracks[user] = tracks_info
        
    return users_playlists_tracks, mygroup

### OBTAIN FINAL TRACKS

In [218]:
def get_final_tracks(users_playlists_tracks, mygroup):
    
    final_tracks = {}
    
    final_track_names = []
    final_track_IDs = []
    final_track_artists = []

    for user in users_playlists_tracks:
        
        # Para cada usuario creo una lista de sus canciones con valores únicos. 
        # Esto lo hago para evitar que un usuario con la misma canción en distintas
        # playlists incluya varias veces la canción en la lista final, dándole mayor 
        # popularidad porque se repite entre sus listas, pero sin que realmente sea 
        # popular entre los distintos perfiles.
        
        user_track_names = []
        user_track_IDs = []
        user_track_artists = []
        
        for playlist in users_playlists_tracks[user]:
            for track in playlist["items"]:
                if track["track"]["name"] not in user_track_names:  # Aquí evito repetición.
                    if track["track"]["album"]["artists"] != []:  
                        artist_id = track["track"]["album"]["artists"][0]["external_urls"]["spotify"][32:]
                        artist_info = sp.artist(artist_id)
                        genre_in_artist = 0
                        if mygroup["users"][user]["genres"] == ["any"]:
                            user_track_names.append(track["track"]["name"])
                            user_track_IDs.append(track["track"]["id"])
                            user_track_artists.append(track["track"]["album"]["artists"][0]["name"])
                        else:
                            for genre in mygroup["users"][user]["genres"]:
                                if genre in artist_info["genres"]:
                                    genre_in_artist += 1
                            if genre_in_artist > 0:
                                user_track_names.append(track["track"]["name"])
                                user_track_IDs.append(track["track"]["id"]) 
                                user_track_artists.append(track["track"]["album"]["artists"][0]["name"])
                            
        for i in range(len(user_track_names)):
            final_track_names.append(user_track_names[i])
            final_track_IDs.append(user_track_IDs[i]) 
            final_track_artists.append(user_track_artists[i])        
            
    
    for i in range(len(final_track_names)):
        final_tracks[final_track_IDs[i]] = [final_track_names[i], final_track_artists[i]]
        
    
    # Hago este shuffle antes de ordenarlos por popularidad para evitar que las canciones con 
    # el mismo número de repeticiones se ordenen según el orden de introducción de los 
    # nombres de usuarios. Lo quiero aleatorio:
    
    shuffle(final_track_IDs)
    final_track_IDs_pop = [item for items, c in Counter(final_track_IDs).most_common() for item in [items] * c]
    
    final_track_IDs_unique = []
    
    for track_ID in final_track_IDs_pop:
        if track_ID not in final_track_IDs_unique:
            final_track_IDs_unique.append(track_ID)
            
    print(final_track_IDs[:10])
    print(final_track_IDs_pop[:10])
    
    return final_tracks, final_track_IDs_unique, final_track_IDs_pop, mygroup

### CREATE DATAFRAME FOR DJ

In [219]:
def create_dataframe(final_tracks, final_track_IDs_unique, final_track_IDs_pop, mygroup):

    tracks = []
    artists = []

    for ID in final_track_IDs_pop:
        if len(tracks) < 100:
            tracks.append(final_tracks.get(ID)[0])
            artists.append(final_tracks.get(ID)[1])

    final_tracks_df = pd.DataFrame(data = [final_track_IDs_pop[:100], tracks, artists],
                                   columns = range(100), 
                                   index = ["ID", "Track", "Artist"]).T
    
    display(final_tracks_df)

    return final_track_IDs_unique, mygroup

### CREATE PLAYLIST ON SPOTIFY

In [220]:
def create_playlist(final_track_IDs_unique, mygroup):

    token = util.prompt_for_user_token(host,
                               "playlist-modify-public",
                               client_id = "{}".format(client_ID),
                               client_secret = "{}".format(client_secret),
                               redirect_uri = "https://example.com/callback/")

    sp = spotipy.Spotify(auth = token)
    final_playlist = sp.user_playlist_create(host, mygroup["new_playlist_name"])

    sp.user_playlist_add_tracks(host, final_playlist["uri"][17:], final_track_IDs_unique[:100])

    final_playlist_url = final_playlist["external_urls"]["spotify"]
    
    print("""
          Here's your new combined playlist: 
          
          {}
          
          ENJOY!
          """.format(final_playlist["external_urls"]["spotify"])
          )
    

### PIPELINE

In [221]:
if __name__ == '__main__':
    
    users_playlists_names_IDs, mygroup = get_user_playlists()
    
    users_playlists_tracks, mygroup = get_tracks_info(users_playlists_names_IDs, mygroup)
    
    final_tracks, final_track_IDs_unique, final_track_IDs, mygroup = get_final_tracks(users_playlists_tracks, mygroup)
    
    final_track_IDs_unique, mygroup = create_dataframe(
        final_tracks, final_track_IDs_unique, final_track_IDs, mygroup)
    
    create_playlist(final_track_IDs_unique, mygroup)
    


            ·················································
            
            sedepa , enter the Spotify playlists you would like to upload, separated by '//'. 
                        
            These are all your playlists: 
    
            ['MAD', 'Sunlight', 'Mana Maoli', 'Talking', 'Chosen one', 'Grandes maestros alemanes del arpa que nacieron en el siglo XX', 'Beatz', 'Interference', 'Movida Milennial', 'Si algún día me convierto en jugador de NBA estos serán mis jams', 'Al Son Club Social', 'Night-light', 'Bonfire.', 'El gusto es mio', 'Palante', 'Wgłębienie', 'Nueva barrica', 'Ace à Paris', 'Chill', "Pugs 'n' hugs", 'Just de Paso', 'Tobillos', 'Niggas Flow', 'retro touch', 'Sounds great', 'Coast Road', 'New Zihuatanejo', 'Mudarse a Zihuatanejo', 'Dormir', 'Baby you and me', 'De ayer y de hoy', 'Más Eñe', 'A little of verguenza, please', 'Correr'] 
            
            
            If you want to submit all, just write "all"
                        
            

,ID,Track,Artist
0,0dWvKc40j9oBcmSpnQcZvg,A Ti Te Ocurre Algo,Leiva
1,0dWvKc40j9oBcmSpnQcZvg,A Ti Te Ocurre Algo,Leiva
2,7rbsQlwPWCnz72GdzVOi9h,Godzilla (feat. Enrique Bunbury & Ximena Sariñ...,Leiva
3,7rbsQlwPWCnz72GdzVOi9h,Godzilla (feat. Enrique Bunbury & Ximena Sariñ...,Leiva
4,4aAfLSx9IthpC3Pw5pNk3E,Como Si Fueras a Morir Mañana,Leiva
5,4aAfLSx9IthpC3Pw5pNk3E,Como Si Fueras a Morir Mañana,Leiva
6,6XX8yvgn17saKL9rcygqez,Pienso en Aquella Tarde,Pereza
7,0Fdg1K74O83ByiiOUpNBq5,El Sueño - Radio Edit,Dennis Cruz
8,5Zpy2HR2vYLjlTQVd81DLH,Paris 1919,John Cale
9,3uSSjnDMmoyERaAK9KvpJR,Too Deep,dvsn



          Here's your new combined playlist: 
          
          https://open.spotify.com/playlist/0kXQqNjclfs97MQ9m0TkIY
          
          ENJOY!
          
